About Dataset
Context
Having seen multiple datasets related to face mask detection on Kaggle, one dataset which stood out contained 3 classes (with mask, without a mask, and wearing mask incorrectly), unfortunately, the dataset was highly imbalanced and uncleaned. So to improve this dataset, images had to be augmented in such a way that each class has an equal distribution of images and removing noisy images which could be considered as outliers. Thus this dataset that I've created is a combination of an existing dataset that has been cleaned and equally distributed across each class.

Content
The dataset contains 3 folders labeled as to which class they belong to. the 3 classes are "with_mask", "withou_mask", and "mask_weared_incorrect". Each folder holds 2994 images of people that belong to such a labeled class.

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d vijaykumar1799/face-mask-detection

 96% 213M/222M [00:02<00:00, 92.2MB/s]
100% 222M/222M [00:02<00:00, 85.8MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/face-mask-detection.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [1]:
import pandas as pd
import numpy as np
import random
import cv2
import os
from shutil import copyfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,Dense,BatchNormalization,Dropout,Flatten
from tensorflow.keras.applications.efficientnet import EfficientNetB1
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,array_to_img,load_img
import warnings
warnings.filterwarnings("ignore")

In [4]:
categories = ['mask_weared_incorrect', 'with_mask', 'without_mask']

In [5]:
data = []

for catigory in categories:
  path = os.path.join("/content/Dataset",catigory)

  label = categories.index(catigory)

  for file in os.listdir(path):

    image_path = os.path.join(path,file)
    image = cv2.imread(image_path)
    image = cv2.resize(image,(224,224))

    data.append([image,label])

In [6]:
random.shuffle(data)#suffle the data

In [7]:
# now saprate the depandent and indipendent features
X = []
y = []

for features,label in data:
  X.append(features)
  y.append(label)

In [8]:
# now check the len
print(len(X))
print(len(y))

8982
8982


In [9]:
# convert in to numpy array
X = np.array(X)
y = np.array(y)

In [10]:
print(X.shape)
print(y.shape)

(8982, 224, 224, 3)
(8982,)


In [11]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets (adjust the test_size as needed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (7185, 224, 224, 3)
X_test shape: (1797, 224, 224, 3)
y_train shape: (7185,)
y_test shape: (1797,)


In [13]:
# Create an instance of ImageDataGenerator with desired augmentations
BATCH_SIZE = 32
traingen = ImageDataGenerator(
    rescale = 1./255,# Normalize pixel values to the range [0, 1]
    width_shift_range=0.2,  # Randomly shift images horizontally by up to 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by up to 20% of the height
    shear_range=0.2,  # Apply shear transformations
    zoom_range=0.2,  # Randomly zoom in or out by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill points outside the boundaries with the nearest pixel value
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values to the range [0, 1]

In [14]:
# Create generators for your training and testing data
train_data = traingen.flow(X_train, y_train, batch_size=BATCH_SIZE)
test_data = test_datagen.flow(X_test, y_test, batch_size=BATCH_SIZE)

In [15]:
# Now prepare the effecient net model
efficientnet = EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3),
)

27018416/27018416 [==============================] - 0s 0us/step


In [18]:
# now check the layers in resnet
def check_layers():
  for layer in efficientnet.layers:
    print(layer.name,layer.trainable)

check_layers()

input_1 True
rescaling True
normalization True
rescaling_1 True
stem_conv_pad True
stem_conv True
stem_bn True
stem_activation True
block1a_dwconv True
block1a_bn True
block1a_activation True
block1a_se_squeeze True
block1a_se_reshape True
block1a_se_reduce True
block1a_se_expand True
block1a_se_excite True
block1a_project_conv True
block1a_project_bn True
block1b_dwconv True
block1b_bn True
block1b_activation True
block1b_se_squeeze True
block1b_se_reshape True
block1b_se_reduce True
block1b_se_expand True
block1b_se_excite True
block1b_project_conv True
block1b_project_bn True
block1b_drop True
block1b_add True
block2a_expand_conv True
block2a_expand_bn True
block2a_expand_activation True
block2a_dwconv_pad True
block2a_dwconv True
block2a_bn True
block2a_activation True
block2a_se_squeeze True
block2a_se_reshape True
block2a_se_reduce True
block2a_se_expand True
block2a_se_excite True
block2a_project_conv True
block2a_project_bn True
block2b_expand_conv True
block2b_expand_bn True
b

In [19]:
def fine_tuening(target_block):
  efficientnet.trainable = True
  set_trainable = False

  for layer in efficientnet.layers:
    if layer.name == target_block:
      set_trainable = True

    if set_trainable:
      layer.trainable = True

    else:
      layer.trainable = False

fine_tuening("block5a_expand_conv")

In [20]:
# efficientnet.trainable = False
check_layers()

input_1 False
rescaling False
normalization False
rescaling_1 False
stem_conv_pad False
stem_conv False
stem_bn False
stem_activation False
block1a_dwconv False
block1a_bn False
block1a_activation False
block1a_se_squeeze False
block1a_se_reshape False
block1a_se_reduce False
block1a_se_expand False
block1a_se_excite False
block1a_project_conv False
block1a_project_bn False
block1b_dwconv False
block1b_bn False
block1b_activation False
block1b_se_squeeze False
block1b_se_reshape False
block1b_se_reduce False
block1b_se_expand False
block1b_se_excite False
block1b_project_conv False
block1b_project_bn False
block1b_drop False
block1b_add False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_dwconv_pad False
block2a_dwconv False
block2a_bn False
block2a_activation False
block2a_se_squeeze False
block2a_se_reshape False
block2a_se_reduce False
block2a_se_expand False
block2a_se_excite False
block2a_project_conv False
block2a_project_bn False
block

In [21]:
efficientnet.summary()

Model: "efficientnetb1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0

In [22]:
# now create the dense layer to train the model on convo base

model = Sequential()

# add effecientnet convolution base
model.add(efficientnet)

# add flatten layer to convert in to 1D array
model.add(Flatten())

# now add dense layer1
model.add(Dense(350,activation="elu",kernel_initializer="he_normal"))
model.add(BatchNormalization())


# now add dense layer2
model.add(Dense(250,activation="elu",kernel_initializer="he_normal"))
model.add(BatchNormalization())

# now add dense layer3
model.add(Dense(100,activation="elu",kernel_initializer="he_normal"))
model.add(BatchNormalization())

# oytput layer
model.add(Dense(3,activation="softmax"))

In [23]:
# get model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb1 (Functional  (None, 7, 7, 1280)        6575239   
 )                                                               
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 350)               21952350  
                                                                 
 batch_normalization (Batch  (None, 350)               1400      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 250)               87750     
                                                                 
 batch_normalization_1 (Bat  (None, 250)               1

In [26]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2,restore_best_weights=True)

In [27]:
# Now compile the model
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics = ["accuracy"]
)

In [ ]:
# Train model
model.fit(
    train_data,
    epochs = 15,
    steps_per_epoch=len(train_data),
    callbacks = [early_stopping],
    validation_data = test_data
)

Epoch 1/15


In [ ]:
model.save("model.h5")  #saving model